# Timetable SA: Simulated Annealing Optimization

This notebook demonstrates how to use the `timetable-sa` Python package for constraint optimization problems.

## Installation

```python
!pip install timetable-sa
```

Or for development:

```python
!pip install -e /path/to/timetable-sa-python
```

## Basic Usage

Let's create a simple optimization problem to demonstrate the API.

In [ ]:
from timetable_sa import SimulatedAnnealing, SAConfig, Constraint, MoveGenerator
import copy

# Define a simple state - just a list of numbers
class SimpleState:
    def __init__(self, values):
        self.values = values
    
    def __repr__(self):
        return f"SimpleState(values={self.values})"

# Create initial state with negative values (violating constraints)
initial_state = SimpleState([-5, -3, -8, -2, -6])
print(f"Initial state: {initial_state}")

### Define Constraints

Constraints can be **hard** (must be satisfied) or **soft** (preferred but not required).

In [ ]:
class MinimumValueConstraint(Constraint):
    """Hard constraint: all values must be at least min_value."""
    
    name = "Minimum Value"
    type = "hard"
    min_value = 0
    
    def __init__(self, min_value=0):
        self.min_value = min_value
        self.name = f"Minimum Value (≥{min_value})"
    
    def evaluate(self, state):
        """Return 1.0 if all values >= min_value, else lower score."""
        min_val = min(state.values) if state.values else 0
        if min_val >= self.min_value:
            return 1.0
        # Return score based on how far we are from the minimum
        deficit = self.min_value - min_val
        return max(0.0, 1.0 - deficit / 10.0)
    
    def get_violations(self, state):
        """Return list of violation descriptions."""
        violations = []
        for i, val in enumerate(state.values):
            if val < self.min_value:
                violations.append(f"Value[{i}]={val} < {self.min_value}")
        return violations

class TargetSumConstraint(Constraint):
    """Soft constraint: prefer sum close to target_sum."""
    
    name = "Target Sum"
    type = "soft"
    weight = 1.0
    target_sum = 0
    
    def __init__(self, target_sum=0, weight=1.0):
        self.target_sum = target_sum
        self.weight = weight
        self.name = f"Target Sum (={target_sum})"
    
    def evaluate(self, state):
        """Return 1.0 if sum equals target, else lower score."""
        current_sum = sum(state.values)
        diff = abs(current_sum - self.target_sum)
        max_possible = sum(abs(v) for v in state.values) if state.values else 1
        return max(0.0, 1.0 - diff / (max_possible + 1))

# Create constraints
constraints = [
    MinimumValueConstraint(min_value=0),  # Hard: values must be >= 0
    TargetSumConstraint(target_sum=20, weight=5.0)  # Soft: prefer sum=20
]

print("Constraints defined:")
for c in constraints:
    print(f"  - {c.name} (type: {c.type})")

### Define Move Generators

Move generators define how to explore the solution space.

In [ ]:
class IncreaseValue(MoveGenerator):
    """Increase a random value by 1."""
    
    name = "Increase Value"
    
    def can_apply(self, state):
        return len(state.values) > 0
    
    def generate(self, state, temperature):
        new_state = SimpleState(copy.deepcopy(state.values))
        if new_state.values:
            import random
            idx = random.randint(0, len(new_state.values) - 1)
            new_state.values[idx] += 1
        return new_state

class DecreaseValue(MoveGenerator):
    """Decrease a random value by 1."""
    
    name = "Decrease Value"
    
    def can_apply(self, state):
        return len(state.values) > 0
    
    def generate(self, state, temperature):
        new_state = SimpleState(copy.deepcopy(state.values))
        if new_state.values:
            import random
            idx = random.randint(0, len(new_state.values) - 1)
            new_state.values[idx] -= 1
        return new_state

class SwapValues(MoveGenerator):
    """Swap two random values."""
    
    name = "Swap Values"
    
    def can_apply(self, state):
        return len(state.values) >= 2
    
    def generate(self, state, temperature):
        new_state = SimpleState(copy.deepcopy(state.values))
        if len(new_state.values) >= 2:
            import random
            i, j = random.sample(range(len(new_state.values)), 2)
            new_state.values[i], new_state.values[j] = new_state.values[j], new_state.values[i]
        return new_state

# Create move generators
move_generators = [
    IncreaseValue(),
    DecreaseValue(),
    SwapValues()
]

print("Move generators defined:")
for g in move_generators:
    print(f"  - {g.name}")

### Configure and Run Optimization

Now let's configure the Simulated Annealing algorithm and run it.

In [ ]:
# Configure the algorithm
config = SAConfig(
    initial_temperature=1000.0,
    min_temperature=0.1,
    cooling_rate=0.995,
    max_iterations=5000,
    hard_constraint_weight=10000.0,
    clone_state=lambda s: SimpleState(copy.deepcopy(s.values)),
    enable_intensification=True,
    reheating_threshold=200,
    max_reheats=3,
)

# Create the optimizer
sa = SimulatedAnnealing(
    initial_state=initial_state,
    constraints=constraints,
    move_generators=move_generators,
    config=config
)

print("Simulated Annealing optimizer created!")
print(f"Configuration: {config.initial_temperature} initial temp, {config.max_iterations} max iterations")

In [ ]:
# Run the optimization
print("\nRunning optimization...")
solution = sa.solve()

# Display results
print("\n" + "="*50)
print("OPTIMIZATION RESULTS")
print("="*50)
print(f"Final state: {solution['state']}")
print(f"Fitness: {solution['fitness']:.4f}")
print(f"Hard violations: {solution['hard_violations']}")
print(f"Soft violations: {solution['soft_violations']}")
print(f"Iterations: {solution['iterations']}")
print(f"Reheats: {solution['reheats']}")
print(f"Final temperature: {solution['final_temperature']:.4f}")

In [ ]:
# Show detailed violations
print("\nConstraint Violations:")
if solution['violations']:
    for v in solution['violations']:
        print(f"  - {v['constraint_name']} ({v['constraint_type']}): {v['description']}")
else:
    print("  No violations!")

# Show operator statistics
print("\nOperator Statistics:")
for name, stats in solution['operator_stats'].items():
    print(f"  {name}: {stats['attempts']} attempts, {stats['improvements']} improvements, "
          f"{stats['success_rate']*100:.1f}% success rate")

## Advanced Configuration

### Tabu Search

Enable Tabu Search to prevent the algorithm from cycling back to previously visited states.

In [ ]:
config_with_tabu = SAConfig(
    initial_temperature=1000.0,
    min_temperature=0.1,
    cooling_rate=0.995,
    max_iterations=5000,
    hard_constraint_weight=10000.0,
    clone_state=lambda s: SimpleState(copy.deepcopy(s.values)),
    
    # Tabu Search configuration
    tabu_search_enabled=True,
    tabu_tenure=50,  # States stay tabu for 50 iterations
    max_tabu_list_size=500,  # Maximum states to remember
    
    # Reheating configuration
    reheating_threshold=150,
    reheating_factor=1.5,
    max_reheats=5,
)

# Run with Tabu Search
sa_tabu = SimulatedAnnealing(initial_state, constraints, move_generators, config_with_tabu)
solution_tabu = sa_tabu.solve()

print("Results with Tabu Search enabled:")
print(f"  Hard violations: {solution_tabu['hard_violations']}")
print(f"  Iterations: {solution_tabu['iterations']}")

## Summary

The `timetable-sa` package provides:

- **Multi-phase optimization**: Phase 1 eliminates hard constraints, Phase 2 optimizes soft constraints
- **Tabu Search**: Prevents cycling
- **Intensification**: Aggressively targets stubborn violations
- **Reheating**: Escapes local minima
- **Adaptive operator selection**: Learns which moves work best

For more information, see the [GitHub repository](https://github.com/emmanuelabayor/timetable-sa).